In [1]:
import os
import sys
import warnings
import pandas as pd
from tqdm.notebook import tqdm

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)

base_path: /home/uoscisai/Experiments/Football/sr-press


In [2]:
from socceraction.data.statsbomb import StatsBombLoader
from statsbombpy import sb
from express.databases import SQLiteDatabase
from express.visualization import plot_action
from express.utils import add_names

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
SBL = StatsBombLoader(root=os.path.join(base_path, "../open-data/data"), getter="local")
competition = sb.competitions()

# HTTPError: 404 Client Error: Not Found for ur
Error_competition = [(11, 90), (7, 235), (7, 108), (44, 107)] # La-Liga 20/21, Ligue1 21/22, Ligue1 22/23 
competition = competition[~competition[["competition_id", "season_id"]].apply(tuple, axis=1).isin(Error_competition)] # HTTPError: 404 Client Error: Not Found for ur
competition = competition[(competition["match_available_360"].notna()) & (competition["competition_gender"] == "male")]
competition

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-07-15T14:15:54.671676,2024-07-15T14:17:00.877356,2024-07-15T14:17:00.877356,2024-07-15T14:15:54.671676
29,43,106,International,FIFA World Cup,male,False,True,2022,2024-05-15T10:23:32.854925,2024-06-11T11:30:50.462448,2024-06-11T11:30:50.462448,2024-05-15T10:23:32.854925
68,55,282,Europe,UEFA Euro,male,False,True,2024,2024-07-15T15:48:50.315500,2024-07-15T15:52:24.778809,2024-07-15T15:52:24.778809,2024-07-15T15:48:50.315500
69,55,43,Europe,UEFA Euro,male,False,True,2020,2024-04-16T12:44:40.558402,2024-04-16T12:47:18.505110,2024-04-16T12:47:18.505110,2024-04-16T12:44:40.558402


In [5]:
games = pd.concat([
    SBL.games(row.competition_id, row.season_id)
    for row in tqdm(competition.itertuples())
]).reset_index(drop=True)

train_games = games[
    ((games['competition_id'] == 9) & (games['season_id'] == 281) & (games['game_day'] <= 17)) | # Bundesliga(1~17 Round)
    ((games['competition_id'] == 43) & (games['season_id'] == 106) & (games['game_day'] <= 2)) | # World Cup(1~2 Round)
    ((games['competition_id'] == 55) & (games['season_id'].isin([282, 43])))                     # UEFA Euro 2020, 2024
]

test_games = games[
    ((games['competition_id'] == 9) & (games['season_id'] == 281) & (games['game_day'] > 17)) | # Bundesliga(18~34 Round)
    ((games['competition_id'] == 43) & (games['season_id'] == 106) & (games['game_day'] > 2))   # World Cup(3~7 Round)
]

print(train_games.shape, test_games.shape)

0it [00:00, ?it/s]

(151, 12) (49, 12)


In [6]:
TRAIN_DB_PATH = os.path.join(base_path, "stores/train_database.sqlite")
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")

train_db = SQLiteDatabase(TRAIN_DB_PATH)
test_db = SQLiteDatabase(TEST_DB_PATH)

print("train_db:", train_db)
print("test_db:", test_db)

train_db: <express.databases.sqlite.SQLiteDatabase object at 0x717de0404640>
test_db: <express.databases.sqlite.SQLiteDatabase object at 0x717de0ac6a70>


In [7]:
for _, game in tqdm(train_games.iterrows(), desc="train_game loading"):
    dataset = {"getter": "local", 
                "competition_id": game["competition_id"], 
                "season_id": game["season_id"], 
                "game_id": game["game_id"],
                "root": os.path.join(base_path, "../open-data/data")}
    
    train_db.import_data(**dataset)

for _, game in tqdm(test_games.iterrows(), desc="test_game loading"):
    dataset = {"getter": "local", 
                "competition_id": game["competition_id"], 
                "season_id": game["season_id"], 
                "game_id": game["game_id"],
                "root": os.path.join(base_path, "../open-data/data")}
    
    test_db.import_data(**dataset)

train_game loading: 0it [00:00, ?it/s]

test_game loading: 0it [00:00, ?it/s]


KeyboardInterrupt



In [ ]:
game_id = 3895202

df_actions = add_names(test_db.actions(game_id)).reset_index()
df_actions.head()

In [ ]:
df_actions[df_actions["original_event_id"] == "c2d74b65-f31d-4d50-a027-fb42fec6ac60"].iloc[:, :13]

In [ ]:
df_actions[df_actions["original_event_id"] == "c2d74b65-f31d-4d50-a027-fb42fec6ac60"].iloc[:, 13:]

In [ ]:
df_actions[(df_actions["type_name"] == "pressing") & (df_actions["result_name"] == "success")]

In [ ]:
df_actions[df_actions["type_name"] == "pressing"].index[62]

In [ ]:
pressure_idx = df_actions[df_actions["type_name"] == "pressing"].index[62]
home_team_id, away_team_id = train_db.get_home_away_team_id(game_id = 3788741)

print(home_team_id, away_team_id)

for idx in range(pressure_idx-10, pressure_idx+2):
    if df_actions.loc[idx]["freeze_frame_360"] is None:
        print("Skip action due to missing freeze frame:",idx)
        plot_action(df_actions.loc[idx], home_team_id=home_team_id, show_visible_area=False)
    else:
        # plot_action: 데이터의 위치 정보는 홈팀은 left->right, 원정팀은 left<-right인 상태이다.
        plot_action(df_actions.loc[idx], home_team_id=home_team_id)